In [ ]:
pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unidecode import unidecode
import zipfile

# **Kaggle dataset**

In [ ]:
!kaggle datasets download -d sbhatti/news-summarization

In [ ]:
with zipfile.ZipFile('news-summarization.zip', 'r') as zip_ref:
    zip_ref.extractall('news-summarization')

In [ ]:
news_data = pd.read_csv("news-summarization/data.csv")

In [ ]:
news_data.head()

In [ ]:
news_data["Content"] = news_data["Content"].apply(
    lambda text: ' '.join([unidecode(token) for token in str(text).split()])
)
news_data["Summary"] = news_data["Summary"].apply(
    lambda text: ' '.join([unidecode(token) for token in str(text).split()])
)

In [ ]:
print(news_data["Content"][0])
print()
print(news_data["Summary"][0])

In [ ]:
lengths_article = news_data["Content"].str.len()
lengths_article.describe()

In [ ]:
news_data = news_data[(lengths_article >= lengths_article.quantile(0.05)) & (lengths_article <= lengths_article.quantile(0.95))]

In [ ]:
pd.DataFrame(news_data["Content"].str.len()).plot(kind='hist', bins=50, edgecolor='black', alpha=0.7, color='blue')
plt.axvline(news_data["Content"].str.len().mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {news_data["Content"].str.len().mean():.2f}')
plt.xlabel("Article Length (tokens)", fontsize=14)
plt.ylabel("")
plt.legend(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
lengths_summary = news_data["Summary"].str.len()
lengths_summary.describe()

In [ ]:
news_data = news_data[(lengths_summary >= lengths_summary.quantile(0.05)) & (lengths_summary <= lengths_summary.quantile(0.95))]

In [ ]:
news_data["Summary"].str.len().describe()

In [ ]:
news_data["Summary"].str.len().plot(kind='hist', bins=50, edgecolor='black', alpha=0.7, color='blue')
plt.axvline(news_data["Summary"].str.len().mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {news_data["Summary"].str.len().mean():.2f}')
plt.xlabel("Summary Length (tokens)", fontsize=14)
plt.ylabel("")
plt.legend(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# **RNN**

In [ ]:
import torch
from torch import nn

_ = torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **LLM Llama 3.2**

In [ ]:
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
import yaml

with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)

hf_token = config["huggingface"]["token"]
login("hf_token")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")